In [0]:
%sql
use catalog dev;
create schema dev.leena;

In [0]:
%sql
create table dev.leena.demo (id int, name string)

In [0]:
%python
(
    spark
    .read
    .csv("/Volumes/dev/leena/volume/csv/financial_dataset.csv",header=True,inferSchema=True)
    .write
    .mode("overwrite")
    .saveAsTable("dev.leena.financial")
    )

In [0]:
df=spark.read.table("dev.leena.financial")

In [0]:
df.filter("Branch='Tokyo'").explain(True)

In [0]:
df.filter("Branch='Tokyo'")

In [0]:
%sql
select * from dev.leena.financial where Branch="Tokyo"

In [0]:
df.display()

In [0]:
%sql
select TransactionType, count(*) as count from dev.leena.financial group by TransactionType

In [0]:
df.withColumnRenamed("TransactionID","transaction_id").withColumnRenamed("CustomerID","customer_id")


In [0]:
df.withColumnsRenamed({"TransactionID":"transaction_id","CustomerID":"customer_id"})


In [0]:
df.show()

In [0]:
from pyspark.sql.functions import round, current_date, lit
df.withColumn("AmountUSD", round("AmountUSD")).withColumn("ingestion_date", current_date()).withColumn("env", lit("dev")).display()


In [0]:
from pyspark.sql.functions import *

In [0]:
(
    spark
    .read
    .json("/Volumes/dev/naval/raw/accounts.json")
    
    )

In [0]:
df.show()

In [0]:
df=spark.read.option("multiline", True).json("/Volumes/dev/naval/raw/accounts.json")

In [0]:
df.show()

In [0]:
df.withColumn("ingestion_date",current_timestamp()).display()

In [0]:
df.withColumnRenamed("account_id","accountid").display()

In [0]:
df.drop("created_timestamp")

In [0]:
df.show()


In [0]:
df.withColumn("nominee_name",col("nominee_details.nominee_name")).withColumn("relationship",col("nominee_details.relationship"))


In [0]:
df.display()


In [0]:
df.write.mode("overwrite").saveAsTable("dev.leena.account")

In [0]:
%sql
select * from dev.leena.account

In [0]:
input_path= "/Volumes/dev/naval/raw/accounts.json"

In [0]:
# reader
df=spark.read.option("multiline",True).json(input_path)


# Transformation
df1=df\
.withColumn("nominee_name",col("nominee_details.nominee_name"))\
.withColumn("relationship",col("nominee_details.relationship"))\
.withColumn("ingestion_date",current_timestamp())\
.drop("nominee_details","created_timestamp")


#Writer
df1.write.mode("overwrite").saveAsTable("dev.leena.account")

In [0]:
%sql
create table dev.leena.medical as 
select *,current_timestamp() as ingestion_date from json.`/Volumes/dev/naval/raw/1.json`